# Researching User Spotify Data
The purpose of this notebook is to explore the data and create the algorithms that will be used in the django views.

### To-Do
- ~Liked tracks should be considered in the algo.~ When a top track matches with a saved track, it should carry more weight--create a metric for this -- idk how, just think about it in the future.
    - 1+ for each time an a track/artist appears, divided by 100
        - Also consider how many times a track shows up in their playlists; it should influence the weight metric. Consider this for artists as well to add more weight to the genres.
        - Also consider their followed artists and playlists.

- Consider getting the user's profile picture and artist picture for front-end purposes

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import plotly.express as px
from time import sleep
import json

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=config['OAuth']['client_id'],
                                               client_secret=config['OAuth']['client_secret'],
                                               redirect_uri='http://localhost/',
                                               scope='user-library-read user-read-playback-state user-read-currently-playing user-top-read'))

## Spotify Functions

In [4]:
# find the user's top artists and return a dictionary that contains a custom artist dictionary and the raw response from the api.
def user_top_artists(sp_token: str, time_range: str):
    artists = []
    response = sp_token.current_user_top_artists(time_range=time_range, limit=50)
    
    for item in response['items']:
        artist = {
            'artist_name': item['name'],
            'artist_id': item['id'],
            'artist_uri': item['uri'],
            'genres': item['genres'],
            'popularity': item['popularity']
            
        }
        artists.append({
            'artist': artist,
            'response': item
        })
    return artists

In [5]:
# get the top tracks from a user
def user_top_tracks(sp_token: str, time_range: str):
    tracks = []
    response = sp_token.current_user_top_tracks(time_range=time_range, limit=50)
    
    for item in response['items']:
        track = {
            'track_name': item['name'],
            'track_id': item['id'],
            'artist_names': [x['name'] for x in item['artists']],
            'artist_ids': [x['id'] for x in item['artists']],
            'artist_uris': [x['id'] for x in item['artists']],
            'duration_ms': item['duration_ms'],
            'explicit': item['explicit'],
            'popularity': item['popularity']
        }
        tracks.append({
            'track': track,
            'response': item
        })
        
    return tracks

In [6]:
# get a user's saved tracks -- limited to 550 tracks
# the limit is due to the amount of time it takes to get the saved tracks without reaching the API rate limit
# the rate limit could be extended if the app was in production and approved by spotify

def get_saved_tracks(sp_token: str):
    offset = 0
    limit = 50
    response = sp_token.current_user_saved_tracks(limit=limit, offset=offset)
    tracks = response['items']

    for i in range(0, 10):
        sleep(1)
        offset += limit
        
        if response['next'] != None or i < 10:
            response = sp_token.current_user_saved_tracks(limit=limit, offset=offset)
            tracks.extend(response['items'])
            i += 1
        else:
            return tracks
        
    return tracks

In [7]:
# get the genres from a list of tracks
# its ok for duplicate genres because we want those duplicates to hold more weight when analyzing them

# there needs to be a way to handle artists that don't have genres listed

def get_genres_from_tracks(sp_token: str, tracks: list):
    
#     response_list = [item['response'] for item in tracks]
    response_list = tracks
    genres_complete = []
    artists_genres = []
    artist_ids = []

    for track in [x['track'] for x in response_list]:
        for artist in track['artists']:
            artist_ids.append(artist['id'])

    try:
        for i in range(0, len(set(artist_ids)), 50):
            batch_artists = artist_ids[i:i+50]
            sleep(1)
            artists = sp_token.artists(batch_artists)
            for artist in artists['artists']:
                genres.append((artist['id'], artist['genres']))
                genres_complete.extend(artist['genres'])

    except Exception as e:
        print(f'ERROR FETCHING GENRES {e}') 

    return (genres, genres_complete)

In [8]:
# def get_artists_from_tracks(sp_token: str, tracks: list):
#     artist_ids = []
#     artists = []
    
#     for i in range(0, len(tracks), )
    

In [9]:
# count the number of times an artist or track occurs across the time-ranges. it returns a dictionary of the track or artist name and the counts.
def count_items(items: list, item_type: str):
    ids = [x[item_type] for x in items]
    for item in items:
#         print(item[item_type])
#         print(ids.count(item[item_type]))
        item['counts'] = ids.count(item[item_type])
    return items

### Saved tracks
This is distinct from the top tracks and makes the tracks and artists lists more robust.

In [10]:
# define the saved track genre list

saved_tracks = get_saved_tracks(sp)

In [11]:
saved_track_dicts = []
for track in saved_tracks:
    saved_track_dicts.append({
        'track_name': track['track']['name'],
        'track_id': track['track']['id'],
        'artist_names': [x['name'] for x in track['track']['artists']],
        'artist_ids': [x['id'] for x in track['track']['artists']],
        'artist_uris': [x['uri'] for x in track['track']['artists']],
        'duration_ms': track['track']['duration_ms'],
        'explicit': track['track']['explicit'],
        'popularity': track['track']['popularity'],
        'added_at': pd.to_datetime(track['added_at']),
    })
    
pd.DataFrame(saved_track_dicts).head()

,track_name,track_id,artist_names,artist_ids,artist_uris,duration_ms,explicit,popularity,added_at
0,Whacko Jacko Steals The Elephant Man's Bones,69xFp8tGO0yEYbhzaXs3Nh,[The Fall of Troy],[5fuQrhMRYMtoO9uOlFad4P],[spotify:artist:5fuQrhMRYMtoO9uOlFad4P],291586,False,22,2024-02-13 00:58:24+00:00
1,The Circus That Has brought Us Back To These N...,0fbHJ5ed7WDfc7buiDpy33,[The Fall of Troy],[5fuQrhMRYMtoO9uOlFad4P],[spotify:artist:5fuQrhMRYMtoO9uOlFad4P],188986,False,26,2024-02-13 00:47:30+00:00
2,From Finish to Starting Line,7cso45ExQPfQn9lYyXV9Ci,[Counterfit],[27w31c5ZkBHHXMlqRGYkJ1],[spotify:artist:27w31c5ZkBHHXMlqRGYkJ1],234693,False,15,2024-02-11 01:08:42+00:00
3,A Piece Of Mind,4FhWqSmp6ROi6QVTgO1oJQ,[Finch],[528t5c0zyuG9beehtth4Za],[spotify:artist:528t5c0zyuG9beehtth4Za],185386,False,33,2024-02-11 00:38:08+00:00
4,Birds,0sXZN04qF8n2Zjgtj3x4r2,"[Bodysync, Ryan Hemsworth, Giraffage]","[34KMxwDAHIvM7Kwt1PcClb, 2CgysNw5B7rFNRtRjQbPZ...","[spotify:artist:34KMxwDAHIvM7Kwt1PcClb, spotif...",209076,False,34,2024-02-10 20:02:04+00:00


### Define artists lists

In [32]:
# define top artists lists by time-range

long_term_artists = user_top_artists(sp, 'long_term')
medium_term_artists = user_top_artists(sp, 'medium_term')
short_term_artists = user_top_artists(sp, 'short_term')

# combine all artists
all_top_artists = [x['artist'] for x in long_term_artists] + [x['artist'] for x in medium_term_artists] + [x['artist'] for x in short_term_artists]

# long_artists_df = pd.DataFrame([x['artist'] for x in long_term_artists])
# medium_artists_df = pd.DataFrame([x['artist'] for x in medium_term_artists])
# short_artists_df = pd.DataFrame([x['artist'] for x in short_term_artists])

In [13]:
long_term_artists[0]

{'artist': {'artist_name': 'Slikback',
  'artist_id': '0NwRAG9DawUqqgur9925fA',
  'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
  'generes': ['african electronic',
   'deconstructed club',
   'experimental techno',
   'grimewave',
   'mandible',
   'singeli'],
  'popularity': 25},
 'response': {'external_urls': {'spotify': 'https://open.spotify.com/artist/0NwRAG9DawUqqgur9925fA'},
  'followers': {'href': None, 'total': 12152},
  'genres': ['african electronic',
   'deconstructed club',
   'experimental techno',
   'grimewave',
   'mandible',
   'singeli'],
  'href': 'https://api.spotify.com/v1/artists/0NwRAG9DawUqqgur9925fA',
  'id': '0NwRAG9DawUqqgur9925fA',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab6761610000e5eb266eedf89f89e330ffcdc25c',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ab67616100005174266eedf89f89e330ffcdc25c',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/ab6761610000f178266eedf8

In [14]:
all_top_artists[0]

{'artist_name': 'Slikback',
 'artist_id': '0NwRAG9DawUqqgur9925fA',
 'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
 'generes': ['african electronic',
  'deconstructed club',
  'experimental techno',
  'grimewave',
  'mandible',
  'singeli'],
 'popularity': 25}

In [33]:
# count of all artists across time-ranges
top_artists_counts = count_items(all_top_artists, 'artist_id')

In [34]:
all_top_artists[0]

{'artist_name': 'Slikback',
 'artist_id': '0NwRAG9DawUqqgur9925fA',
 'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
 'generes': ['african electronic',
  'deconstructed club',
  'experimental techno',
  'grimewave',
  'mandible',
  'singeli'],
 'popularity': 25,
 'counts': 2}

In [18]:
long_term_artists[0]

{'artist': {'artist_name': 'Slikback',
  'artist_id': '0NwRAG9DawUqqgur9925fA',
  'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
  'generes': ['african electronic',
   'deconstructed club',
   'experimental techno',
   'grimewave',
   'mandible',
   'singeli'],
  'popularity': 25,
  'counts': 2},
 'response': {'external_urls': {'spotify': 'https://open.spotify.com/artist/0NwRAG9DawUqqgur9925fA'},
  'followers': {'href': None, 'total': 12152},
  'genres': ['african electronic',
   'deconstructed club',
   'experimental techno',
   'grimewave',
   'mandible',
   'singeli'],
  'href': 'https://api.spotify.com/v1/artists/0NwRAG9DawUqqgur9925fA',
  'id': '0NwRAG9DawUqqgur9925fA',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab6761610000e5eb266eedf89f89e330ffcdc25c',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ab67616100005174266eedf89f89e330ffcdc25c',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/ab6761610

In [19]:
top_artists_counts[0]

{'artist_name': 'Slikback',
 'artist_id': '0NwRAG9DawUqqgur9925fA',
 'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
 'generes': ['african electronic',
  'deconstructed club',
  'experimental techno',
  'grimewave',
  'mandible',
  'singeli'],
 'popularity': 25,
 'counts': 2}

In [35]:
# create a distinct list of the artists
existing_artists = []
all_artists_distinct = []
for artist in top_artists_counts:
    if artist['artist_id'] not in existing_artists:
        existing_artists.append(artist['artist_id'])
        all_artists_distinct.append(artist)
    else:
        continue

In [36]:
all_artists_distinct[0]

{'artist_name': 'Slikback',
 'artist_id': '0NwRAG9DawUqqgur9925fA',
 'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
 'generes': ['african electronic',
  'deconstructed club',
  'experimental techno',
  'grimewave',
  'mandible',
  'singeli'],
 'popularity': 25,
 'counts': 2}

In [37]:
### get all the artists from the saved tracks and count them
saved_artists = []
for track in saved_track_dicts:
    artists = [x for x in track['artist_ids']]
    for artist in artists:
        if artist not in existing_artists:
            saved_artists.append({
                'artist_id': artist,
                'counts': 1
            })
        else:
            for d in all_artists_distinct:
                if d['artist_id'] == artist:
                    d['counts'] += 1
                    break

In [38]:
all_artists_distinct[0]

{'artist_name': 'Slikback',
 'artist_id': '0NwRAG9DawUqqgur9925fA',
 'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
 'generes': ['african electronic',
  'deconstructed club',
  'experimental techno',
  'grimewave',
  'mandible',
  'singeli'],
 'popularity': 25,
 'counts': 8}

In [39]:
saved_artists[0]

{'artist_id': '5fuQrhMRYMtoO9uOlFad4P', 'counts': 1}

In [26]:
len(all_artists_distinct)

84

In [48]:
# iterate through the saved artists that aren't in the top artists and get their complete data

saved_artists_ids = [x['artist_id'] for x in saved_artists]

for i in range(0, len(saved_artists_ids), 50):
    print(i)
    batch_artists = saved_artists_ids[i:i+50]
    sleep(1)
    response = sp.artists(batch_artists)
    for artist in response['artists']:
        print(f"Artist ID: {artist['id']}")
        
        for d in saved_artists:
            if d['artist_id'] == artist['id']:
                print(f'Saved ID: {d["artist_id"]}')
                print('ID MATCH')
                d['artist_name'] = artist['name']
                d['artist_uri'] = artist['uri']
                d['genres'] = artist['genres']
                d['popularity'] = artist['popularity']
                d['counts'] += 1
                all_artists_distinct.append(d)
                break
            else:
                continue
                

0
Artist ID: 5fuQrhMRYMtoO9uOlFad4P
Saved ID: 5fuQrhMRYMtoO9uOlFad4P
ID MATCH
Artist ID: 5fuQrhMRYMtoO9uOlFad4P
Saved ID: 5fuQrhMRYMtoO9uOlFad4P
ID MATCH
Artist ID: 27w31c5ZkBHHXMlqRGYkJ1
Saved ID: 27w31c5ZkBHHXMlqRGYkJ1
ID MATCH
Artist ID: 34KMxwDAHIvM7Kwt1PcClb
Saved ID: 34KMxwDAHIvM7Kwt1PcClb
ID MATCH
Artist ID: 2CgysNw5B7rFNRtRjQbPZ9
Saved ID: 2CgysNw5B7rFNRtRjQbPZ9
ID MATCH
Artist ID: 4kubsO16bEfCADaVUyoYb6
Saved ID: 4kubsO16bEfCADaVUyoYb6
ID MATCH
Artist ID: 56WzaWPSOfPePuPdfHBYr5
Saved ID: 56WzaWPSOfPePuPdfHBYr5
ID MATCH
Artist ID: 0Qur5LqqAVVunjY2PnjhUp
Saved ID: 0Qur5LqqAVVunjY2PnjhUp
ID MATCH
Artist ID: 26Z0ZxMY2uzimneFbrNuSY
Saved ID: 26Z0ZxMY2uzimneFbrNuSY
ID MATCH
Artist ID: 56huNdCA3s7tthaMNhIXLU
Saved ID: 56huNdCA3s7tthaMNhIXLU
ID MATCH
Artist ID: 6JwRFnmMxmWcYribpIJbcS
Saved ID: 6JwRFnmMxmWcYribpIJbcS
ID MATCH
Artist ID: 2PxnKk0fTNgMzm5pY6tINL
Saved ID: 2PxnKk0fTNgMzm5pY6tINL
ID MATCH
Artist ID: 6gwlllidzu0wkRNkXKGDfG
Saved ID: 6gwlllidzu0wkRNkXKGDfG
ID MATCH
Artist ID:

Artist ID: 1Lqdsv7Ff4GNq9PM3Yd0vi
Saved ID: 1Lqdsv7Ff4GNq9PM3Yd0vi
ID MATCH
Artist ID: 3SR0KKxXYspg7mJwSUsGBb
Saved ID: 3SR0KKxXYspg7mJwSUsGBb
ID MATCH
Artist ID: 467M2s2YxXdlL2ZpDUNL3A
Saved ID: 467M2s2YxXdlL2ZpDUNL3A
ID MATCH
Artist ID: 7lmvHeAJ7CIhXCdLKjm7VL
Saved ID: 7lmvHeAJ7CIhXCdLKjm7VL
ID MATCH
Artist ID: 09RKiiT2NAJ7qYoAGOYu9w
Saved ID: 09RKiiT2NAJ7qYoAGOYu9w
ID MATCH
Artist ID: 4Ase9pfG4FCMoiuyRduc8k
Saved ID: 4Ase9pfG4FCMoiuyRduc8k
ID MATCH
Artist ID: 6bAM7jeIX4pI5lZ0QoSZjt
Saved ID: 6bAM7jeIX4pI5lZ0QoSZjt
ID MATCH
Artist ID: 2E701AAAlg7LthbISEZv0N
Saved ID: 2E701AAAlg7LthbISEZv0N
ID MATCH
Artist ID: 4nygSGQAUjpauYKxzqm8MS
Saved ID: 4nygSGQAUjpauYKxzqm8MS
ID MATCH
Artist ID: 4q3ewBCX7sLwd24euuV69X
Saved ID: 4q3ewBCX7sLwd24euuV69X
ID MATCH
Artist ID: 26dSoYclwsYLMAKD3tpOr4
Saved ID: 26dSoYclwsYLMAKD3tpOr4
ID MATCH
Artist ID: 67OokTsDsLUvJI6oIxCigq
Saved ID: 67OokTsDsLUvJI6oIxCigq
ID MATCH
Artist ID: 5TUa95aB5Vu2CzwCnZd6t0
Saved ID: 5TUa95aB5Vu2CzwCnZd6t0
ID MATCH
Artist ID: 5

Artist ID: 3YrAMtgGeFZCc4JCQI9mfr
Saved ID: 3YrAMtgGeFZCc4JCQI9mfr
ID MATCH
Artist ID: 67kJ7VKxNECnr33Y1BJEoO
Saved ID: 67kJ7VKxNECnr33Y1BJEoO
ID MATCH
Artist ID: 56huNdCA3s7tthaMNhIXLU
Saved ID: 56huNdCA3s7tthaMNhIXLU
ID MATCH
Artist ID: 3BQpGPX8wEhlKrICJuCmBd
Saved ID: 3BQpGPX8wEhlKrICJuCmBd
ID MATCH
Artist ID: 6KgydpcCuiFaWZQvCA50YZ
Saved ID: 6KgydpcCuiFaWZQvCA50YZ
ID MATCH
Artist ID: 59e7YxjDTqYuyxi0kTt4fL
Saved ID: 59e7YxjDTqYuyxi0kTt4fL
ID MATCH
Artist ID: 2iyLphVMYwRzaMn9hwHqDf
Saved ID: 2iyLphVMYwRzaMn9hwHqDf
ID MATCH
Artist ID: 1TtJ8j22Roc24e2Jx3OcU4
Saved ID: 1TtJ8j22Roc24e2Jx3OcU4
ID MATCH
Artist ID: 7FFwJQ58hAy7PMo4lUBW96
Saved ID: 7FFwJQ58hAy7PMo4lUBW96
ID MATCH
Artist ID: 5dMa4Q6mkH9767IgmuGuAu
Saved ID: 5dMa4Q6mkH9767IgmuGuAu
ID MATCH
Artist ID: 5kNGExA70Z5twMjo7mFYrW
Saved ID: 5kNGExA70Z5twMjo7mFYrW
ID MATCH
Artist ID: 2URzewPhxn6Si52NkbYxIQ
Saved ID: 2URzewPhxn6Si52NkbYxIQ
ID MATCH
Artist ID: 6YSHFvanPHtLLb6XEYM94z
Saved ID: 6YSHFvanPHtLLb6XEYM94z
ID MATCH
Artist ID: 1

Artist ID: 2zdyZ3Dk59W2RKwAvGioLp
Saved ID: 2zdyZ3Dk59W2RKwAvGioLp
ID MATCH
Artist ID: 5ZPYeVqoWNuukwfarvkyJX
Saved ID: 5ZPYeVqoWNuukwfarvkyJX
ID MATCH
Artist ID: 7lU8Gtn7moZmPqqu4oPkEh
Saved ID: 7lU8Gtn7moZmPqqu4oPkEh
ID MATCH
Artist ID: 2Mpw0nGvpyFqhdqdhNPFsg
Saved ID: 2Mpw0nGvpyFqhdqdhNPFsg
ID MATCH
Artist ID: 0YZFY3TMKjMskbxnYzwT1g
Saved ID: 0YZFY3TMKjMskbxnYzwT1g
ID MATCH
Artist ID: 1o8nCox6ggFAOMN0vZBj6b
Saved ID: 1o8nCox6ggFAOMN0vZBj6b
ID MATCH
Artist ID: 4ClVQWxbbuA2I7Yuyeu5of
Saved ID: 4ClVQWxbbuA2I7Yuyeu5of
ID MATCH
Artist ID: 4ioovQ90tEAnJiQryAtkXl
Saved ID: 4ioovQ90tEAnJiQryAtkXl
ID MATCH
Artist ID: 6SGVSqPxYUYa885J3dtXQu
Saved ID: 6SGVSqPxYUYa885J3dtXQu
ID MATCH
Artist ID: 4J4bsLS3aMvdGzv0tJTB8t
Saved ID: 4J4bsLS3aMvdGzv0tJTB8t
ID MATCH
Artist ID: 1s7yEBXZWlCUwSQQ4Jljnc
Saved ID: 1s7yEBXZWlCUwSQQ4Jljnc
ID MATCH
Artist ID: 2eKOBeqPB9R7yrt5X7P8ZR
Saved ID: 2eKOBeqPB9R7yrt5X7P8ZR
ID MATCH
Artist ID: 30F64wQIHvLiFTGaNZ73nU
Saved ID: 30F64wQIHvLiFTGaNZ73nU
ID MATCH
Artist ID: 1

In [49]:
len(all_artists_distinct)

639

In [50]:
all_artists_distinct[0]

{'artist_name': 'Slikback',
 'artist_id': '0NwRAG9DawUqqgur9925fA',
 'artist_uri': 'spotify:artist:0NwRAG9DawUqqgur9925fA',
 'generes': ['african electronic',
  'deconstructed club',
  'experimental techno',
  'grimewave',
  'mandible',
  'singeli'],
 'popularity': 25,
 'counts': 8}

In [51]:
pd.DataFrame(all_artists_distinct)

,artist_name,artist_id,artist_uri,generes,popularity,counts,genres
0,Slikback,0NwRAG9DawUqqgur9925fA,spotify:artist:0NwRAG9DawUqqgur9925fA,"[african electronic, deconstructed club, exper...",25.0,8,NaN
1,Drumcorps,6EjKG35nRHw9t8ypwDunOB,spotify:artist:6EjKG35nRHw9t8ypwDunOB,"[breakcore, cybergrind]",27.0,4,NaN
2,Deftones,6Ghvu1VvMGScGpOUJBAHNH,spotify:artist:6Ghvu1VvMGScGpOUJBAHNH,"[alternative metal, nu metal, rap metal, rock,...",78.0,8,NaN
3,Filth is Eternal,0BqcFG3uqS8l59OsCtIiH0,spotify:artist:0BqcFG3uqS8l59OsCtIiH0,"[american grindcore, modern hardcore]",18.0,13,NaN
4,AFI,19I4tYiChJoxEO5EuviXpz,spotify:artist:19I4tYiChJoxEO5EuviXpz,"[alternative metal, nu metal, pop punk, post-g...",60.0,5,NaN
...,...,...,...,...,...,...,...
634,Cult Of Luna,7E7fJJpdVgr1F3pfAfRtHe,spotify:artist:7E7fJJpdVgr1F3pfAfRtHe,NaN,35.0,3,"[atmospheric post-metal, atmospheric sludge, d..."
635,Julie Christmas,4rHZDNrVu2XkdO7eysOHei,spotify:artist:4rHZDNrVu2XkdO7eysOHei,NaN,25.0,3,"[gaian doom, post-metal]"
636,julie,3VNzWLePg9jTvQ2ximYOzW,spotify:artist:3VNzWLePg9jTvQ2ximYOzW,NaN,52.0,2,[grungegaze]
637,Planning For Burial,4nhaUpMjrBW8vywsP2yzzD,spotify:artist:4nhaUpMjrBW8vywsP2yzzD,NaN,33.0,2,"[american shoegaze, doomgaze, nu gaze, one-per..."


### Define tracks lists

In [ ]:
# define top tracks lists

long_term_tracks = user_top_tracks(sp, 'long_term')
medium_term_tracks = user_top_tracks(sp, 'medium_term')
short_term_tracks = user_top_tracks(sp, 'short_term')

all_top_tracks = [item['track'] for item in long_term_tracks] + [item['track'] for item in medium_term_tracks] + [item['track'] for item in short_term_tracks]

In [ ]:
all_top_tracks[0]

In [ ]:
# counts of all tracks across time-ranges
all_tracks_counts = count_items(all_top_tracks, 'track_name')
all_tracks_counts

In [ ]:
pd.DataFrame(all_top_tracks)['artist_names'][0][0]

### Genre Counts

In [ ]:
# define the genre list from time ranges

long_genres = get_genres_from_tracks(sp, long_term_tracks)
medium_genres = get_genres_from_tracks(sp, medium_term_tracks)
short_genres = get_genres_from_tracks(sp, short_term_tracks)

In [ ]:
# define the top all-time genres
long_series = pd.Series(data=long_genres)
long_counts = long_series.value_counts()[:10]

# define the top ten genres from the past 6 months
medium_series = pd.Series(data=medium_genres)
medium_counts = medium_series.value_counts()[:10]

# define the top ten genres from the past 4 weeks
short_series = pd.Series(data=short_genres)
short_counts = short_series.value_counts()[:10]

In [ ]:
# define the time-range dataframe for genres
genre_count_df = pd.DataFrame({'all_time_genres': long_counts, '6month_genres': medium_counts, '4week_genres': short_counts})
genre_count_df.fillna(0, inplace=True)
genre_count_df = genre_count_df.astype(int, copy=True)

In [ ]:
# create a series and count
all_genres_series = pd.Series(data=all_genres)
all_genres_counts = all_genres_series.value_counts()[:25]
all_genres_counts

In [ ]:
# define the genre list from the saved tracks

saved_track_genres = get_genres_from_tracks(sp, saved_tracks)

In [ ]:
# combine all the genres
all_genres = saved_track_genres + long_genres + medium_genres + short_genres
len(all_genres)

## Track Features
Mapping out the track features of all the songs to create visualization of the average/median features for each time-frame.
Including saved tracks.

Break down the features based on the time frames and a combined features.

In [ ]:
# long_track_ids = [x['id'] for x in top_tracks['long']]
# medium_track_ids = [x['id'] for x in top_tracks['medium']]
# short_track_ids = [x['id'] for x in top_tracks['short']]

def get_features(token, tracks):
    key_removal = ['type', 'uri', 'track_href', 'analysis_url']
    track_dir = list()
    
    for track in tracks:
        track_dir.append({
            'track_id': track['id'],
            'name': track['name'],
            'artist': track['name']
        })
        
    tracks_df = pd.DataFrame(track_dir)
    features = token.audio_features([x['track_id'] for x in track_dir])
    features = [{key: value for key, value in d.items() if key not in key_removal} for d in features]
    
    audio_feature_mapping = {item['id']: item for item in features}
    feature_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness','liveness',
                   'valence', 'tempo', 'duration_ms', 'time_signature']
    
    for feature in feature_cols:
        tracks_df[feature] = tracks_df['track_id'].map(lambda x: audio_feature_mapping[x][feature])
        
    return tracks_df

In [ ]:
tracks_df = get_features(sp, top_tracks['long'])
tracks_df.head()

In [ ]:
tracks_df['danceability'].describe()

In [ ]:
dance_mean = tracks_df['danceability'].mean()
dance_std = tracks_df['danceability'].std()
dance_zscore = (tracks_df['danceability'] - dance_mean) / dance_std

print(dance_mean)
print(dance_std)
print(tracks_df['danceability'].min())
print(tracks_df['danceability'].max())

In [ ]:
tracks_df.describe()

In [ ]:
dist_df = tracks_df.drop(columns = ['track_id','name', 'artist']).describe()

# User EDA
This is where all of the user data will be combined into one DataFrame and be analyzed. I'll look at:
- Distribution of genres and track features
    - Grouped by time ranges
- Counts of artists and tracks across all time ranges and saved tracks
    - Top tracks and artists from All-time, 6 months, and 4 weeks
- Top tracks from specific albums
- Distribution of artist and track popularity
    - Grouped by time range
- Distribution of explicit tracks
- Distribution of track duration
- Distribution of the custom weights (counts)
    - This could be used to cluster the features of tracks within the median/mean weight

### Visualizations

In [ ]:
fig = px.bar(
    df,
    labels={
        'index':'Genres', 
        'value':'Number of Tracks',
        'variable':'Time Range',
    },
    title='Most Listened Genres',
)
# fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.update_traces(name='All-time', selector={'name':'long_tracks'})
fig.update_traces(name='Past 6 Months', selector={'name':'medium_tracks'})
fig.update_traces(name='Past 4 Weeks', selector={'name':'short_tracks'})

fig.show()

## Genre Features
- Group song features based on genre and apply a heat map. Perform various analysis to find any correlations.
- Compare the features of the individual track compared to the genre features.
- Create the following functions:
    - Collect features and return a list of dictionaries for those features
    - Input the list of features and analyze them (cluster? median?) and return list of the new values

# Create the model

- Try clustering the track features and genre features.
- Maybe make a simple neural net using weights from the custom TBD metric.
    - Give weights to tracks, artists, and genres